<a href="https://colab.research.google.com/github/ShenshunYao/NLP-Course-in-McGill/blob/master/NL2DS_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Space Model of Word Meaning
## Shenshun Yao 260709204
The goal of this assignment is to make you familiar with vector space model of word meaning. You may reuse some of functions you coded in Assignment 1. 

### Warning: This assignment may take substantial time to run if you are not optimizing your code. Make sure you have plenty of time to run if you are new to programming.

Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this problem set to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with current events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 



Run the below code snippet. It will generate a URL which generates an authorization code.* Enter it below to give Colab access to your Google drive. 

*Copy function may not work. If so, manually copy the authorization code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


When you run the `ls` command below, you should see the files in the tweets folder.




In [ ]:
!ls "/content/drive/My Drive/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


In [ ]:
# let's read tweets. These tweets are already tokenized and cleaned (Assignment 1)
tweets = open("/content/drive/My Drive/tweets/covid-tweets-2020-08-10-2020-08-21.tokenized.txt", "r").read().split("\n")
tweets = [tweet.split() for tweet in tweets]

print(len(tweets))

312878


## Problem 1: Word space model

Compute the most important context words of `ventilator`. Use Pointwise Mutual Information (PMI) to rank the context words (Refer to Lecture 4).

We define context as up to 3 words to the left and 3 words to the right. Ignore stop words and words that do not start with [a-z#]. Also ignore words that are not in the top 1000 frequent words.

These context words define the dimensions of the vector space model. Represent each word as a vector (dictionary/counter) of context words with PMI as the importance of the context word. Print the top 20 context words for each.

This is the sample output I got for `ventilator`. Your numbers need not match mine but the ranked list should look close to what I have.

```
[('put', 18.280538283196606), ('wearing', 17.587373569812726), ('even', 17.58651933524197), ('like', 17.402738298715878), ('covid', 17.172590097063086), ('patients', 16.894419647496004), ('use', 16.894298589380956), ('die', 16.89426559608771), ('days', 16.89415252713107), ('needed', 16.489137134110106), ('month', 16.48907033839664), ('weeks', 16.488913820220848), ('away', 16.48879303327717), ('week', 16.488739054051933), ('person', 16.488678720881293), ('good', 16.488160838026904), ('deaths', 16.487822204799755), ('go', 16.487564042558112), ('would', 16.48707075078768), ('one', 16.48706217686235), ('get', 16.486565870239033)]
```



Let's first load stop words.

In [ ]:
stop_words = set()
def load_stop_words():
  words = open("/content/drive/My Drive/tweets/stop_words.txt", "r").read().split("\n")
  for word in words:
    stop_words.add(word.strip())

load_stop_words()
print(len(stop_words))

134


In [ ]:
import re
from collections import Counter
pattern = re.compile(r"^[a-z#].*")

In [ ]:
def clean_a_tweet(tweet,center):
  cleaned_tweet = []
  for word in tweet:
    if word == center or (pattern.match(word) and word not in stop_words):
      cleaned_tweet.append(word)
  return cleaned_tweet

In [ ]:
def keep_topwords(tweet, center, topwords):
  tweet_keepTopwords = []
  for word in tweet:
    if word == center or word in topwords:
      tweet_keepTopwords.append(word)
  return tweet_keepTopwords

In [ ]:
def buildContext(tweet,center,pre,post):
  for i,w in enumerate(tweet):
    if w == center:
      vector = [tweet[i + k] for k in range(-pre, post+1) if 0 <= (i + k) < len(tweet)]
      if vector != []:
        return vector

Let's build the word vectors

In [ ]:
from itertools import combinations
def countUnigram(tweets):
  res = Counter()
  for tweet in tweets:
    for x in tweet:
      res[x] += 1
  return res

def countBigram(tweets):
  res = Counter()
  for tweet in tweets:
    for x, y in map(sorted, combinations(tweet, 2)):
        res[(x, y)] += 1
  return res

In [ ]:
cx = countUnigram(tweets)
cxy = countBigram(tweets)

In [ ]:
N = sum(cx.values())

In [ ]:
from math import log2
PMI_dict = Counter()
for (x, y), n in cxy.items():
    ppmi = log2((n / N) / (cx[x] / N) / (cx[y] / N))
    PMI_dict[(x,y)] = ppmi

In [ ]:
from math import log2

def PMI(word1, word2):
  # You have to store frequencies of individual words and (word, context word) 
  # pairs to compute this. You can compute them beforehand in order to avoid 
  # counting every time when this function is called.
  # Write your code
  if (word1,word2) in PMI_dict.keys():
    return PMI_dict[(word1,word2)]  
  elif (word2,word1) in PMI_dict.keys():
    return PMI_dict[(word2,word1)]

  
def build_word_vector(word):
  # Write your code
  contexts = []
  for tweet in tweets:
    context = buildContext(tweet,word,3,3)
    if context is not None:
      contexts.append(context)
  cleaned_tweets = [clean_a_tweet(tweet,word) for tweet in tweets]
  cleaned_contexts = [clean_a_tweet(context,word) for context in contexts]
  topwords = list(map(lambda x: x[0], Counter(x for tweet in cleaned_tweets for x in set(tweet)).most_common(1000)))
  final_contexts = [keep_topwords(context,word,topwords) for context in cleaned_contexts]
  dict = {}
  for context in final_contexts:
    for element in context:
      if element != word:
        pmi = PMI(word, element)
        dict[element] = pmi
  return dict

def print_top_dimensions(word_vector, n):
  # print top n dimensions sorted in the order of importance.
  print (Counter(word_vector).most_common(n))

v1 = build_word_vector('ventilator')
print_top_dimensions(v1, 20) # print top 20 dimensions along with their weights


[('patient', 10.015404592729878), ('hospital', 8.658607886302383), ('patients', 8.347521351062447), ('k', 7.8989751013209855), ('put', 7.78973786526773), ('enjoy', 7.748305367528045), ('week', 7.688700922845756), ('wearing', 7.656846138567409), ('top', 7.626137162307978), ('brain', 7.575049274693839), ('end', 7.462851842808969), ('weeks', 7.432231951047436), ('died', 7.338278971032686), ('increase', 7.223404257292791), ('die', 7.219919118958893), ('spent', 7.216329148436841), ('days', 7.180218496199567), ('tried', 7.143470720095273), ('month', 7.11959162455242), ('ok', 7.115714451993435)]


## Problem 2: Compute the word similarity between words using Cosine Similarity.

Compute cosine similarity between the following pair of words: 
```
('ventilator', 'covid-19')
('ventilator', 'lockdown')
('ventilator', 'mask')
('ventilator', 'ppe')
```

Outputs of my code are:
```
('ventilator', 'covid-19') 0.17076006036635358
('ventilator', 'lockdown') ...
('ventilator', 'mask') 0.19229601085517933
('ventilator', 'ppe') ...
```


In [ ]:
import numpy as np
def cosine_similarity(vector1, vector2):
  # write your code
  v1 = list(vector1.values())
  v2 = list(vector2.values())
  if len(v1) < len(v2):
    v1.extend([0] * (len(v2) - len(v1)))
  elif len(v2) < len(v1):
    v2.extend([0] * (len(v1) - len(v2)))
  a = np.array(v1)
  b = np.array(v2)
  dot = np.dot(a, b)
  norma = np.linalg.norm(a)
  normb = np.linalg.norm(b)
  cos = dot / (norma * normb) 
  return cos

ventilator = build_word_vector('ventilator')
covid19 = build_word_vector('covid-19')
lockdown = build_word_vector('lockdown')
mask = build_word_vector('mask')
ppe = build_word_vector('ppe')

print(('ventilator', 'covid-19'), cosine_similarity(ventilator, covid19))
print(('ventilator', 'lockdown'), cosine_similarity(ventilator, lockdown))
print(('ventilator', 'mask'), cosine_similarity(ventilator, mask))
print(('ventilator', 'ppe'), cosine_similarity(ventilator, ppe))

('ventilator', 'covid-19') 0.3454885872592941
('ventilator', 'lockdown') 0.3501982721499012
('ventilator', 'mask') 0.37104601269173726
('ventilator', 'ppe') 0.5622871601587083


## Problem 3: What can you tell about these words from the similarities?

1. `ventilator` when compared with `covid-19, lockdown, mask, ppe`
2. `pandemic` when compared with `covid-19, lockdown, mask, ppe`
3. `president` compared with `trump, biden`
4. `trudeau` compared with `trump, biden`



In [ ]:
def compare_similarity(word_a, word_list):
  word_vector_a = build_word_vector(word_a)
  for word_b in word_list:
    word_vector_b = build_word_vector(word_b)
    print((word_a, word_b), cosine_similarity(word_vector_a, word_vector_b))

In [ ]:
compare_similarity('ventilator', ['covid-19', 'lockdown', 'mask', 'ppe'])
compare_similarity('pandemic', ['covid-19', 'lockdown', 'mask', 'ppe'])
compare_similarity('president', ['trump', 'biden'])
compare_similarity('trudeau', ['trump', 'biden'])

('ventilator', 'covid-19') 0.3454885872592941
('ventilator', 'lockdown') 0.3501982721499012
('ventilator', 'mask') 0.37104601269173726
('ventilator', 'ppe') 0.5622871601587083
('pandemic', 'covid-19') 0.9802455806190478
('pandemic', 'lockdown') 0.9651174342597776
('pandemic', 'mask') 0.9542809248387079
('pandemic', 'ppe') 0.6031245551902967
('president', 'trump') 0.908315004755399
('president', 'biden') 0.9357806765987713
('trudeau', 'trump') 0.4850811155587289
('trudeau', 'biden') 0.5292655869804244


After computing the cosine similarity between these four group of words. We found that in the first group, the similarites between these words are not high since they have different semantic meaning in the tweets. On the other hand, the words in the second group mostly have high similarities with each other which means that they are highly relevant in some liguistic senses. Moreover, we know that both 'trump' and 'biden' are highly relevant to 'president'. 

# Let's play with word2vec

First let's load word2vec. I am using [gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html) but feel free to use any libraries or tools.

In [ ]:
from gensim.models import KeyedVectors

EMBEDDING_FILE = '/content/drive/My Drive/tweets/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

## Problem 4: Compute the top 5 similar words of `ventilator` using word2vec?

In [ ]:
# Write your code here
print(word2vec.most_similar(positive=['ventilator'],topn=5))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('respirator', 0.7864563465118408), ('mechanical_ventilator', 0.7063840627670288), ('intensive_care', 0.6809945702552795), ('ventilators', 0.6683582067489624), ('breathing_tube', 0.6665509343147278)]


# Problem 5: Word analogy

If I told you the plural of `car` is `cars`, can you automatically find the plural of `hyothesis` and `man` using word2vec.

Similarly, if I told you a newborn `dog` is called `puppy`, can you automatically find what are the newborn words of `cat` and `sheep` using word2vec?


In [ ]:
# Write your code here.
print(word2vec.most_similar(positive=['hypothesis','cars'],negative=['car'],topn=5))
print(word2vec.most_similar(positive=['man','cars'],negative=['car'],topn=5))
print(word2vec.most_similar(positive=['cat','puppy'],negative=['dog'],topn=5))
print(word2vec.most_similar(positive=['sheep','puppy'],negative=['dog'],topn=5))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hypotheses', 0.6453900337219238), ('theory', 0.5974409580230713), ('theories', 0.566754162311554), ('postulate', 0.5027041435241699), ('supposition', 0.500673770904541)]
[('men', 0.5470781326293945), ('woman', 0.5195947885513306), ('boy', 0.48992404341697693), ('teenager', 0.4598117470741272), ('guy', 0.44635331630706787)]
[('kitten', 0.7634989619255066), ('puppies', 0.7110899090766907), ('pup', 0.6929494142532349), ('kittens', 0.6888390779495239), ('cats', 0.6796488761901855)]
[('lambs', 0.7109252214431763), ('ewes', 0.6476837396621704), ('cattle', 0.6313712000846863), ('cows', 0.6276622414588928), ('calves', 0.6254477500915527)]
